## Topic modelling for the Funda dataset 

### Introduction

This project is executed for the course of AI for Business, at the Amsterdam School of Applied Sciences in 2022. The purpose of the research is to to build and evaluate machine learning approaches to predict the price of a  house and the time to sale of a house. In this notebook, the description of all the house listings on Funda will be split into multiple topic categories based topic modelling. The project is conducted and combined by Pim Doosje (500800438) and Joel Zelle (500763451).

### Importing the dataset and libraries

In [2]:
import pandas as pd
import numpy as np
import spacy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
import pickle
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
import re

In [3]:
# importing file and create a dataframe
df_original = pd.read_csv('housing_data.csv')
# Show first 5 rows
df_original.head()

,globalId,publicatieDatum,postcode,koopPrijs,volledigeOmschrijving,soortWoning,categorieObject,bouwjaar,indTuin,perceelOppervlakte,kantoor_naam_MD5hash,aantalKamers,aantalBadkamers,energielabelKlasse,globalId.1,oppervlakte,datum_ondertekening
0,4388064,2018-07-31,1774PG,139000.0,"Ruimte, vrijheid, en altijd het water en de we...",<{woonboot}> <{vrijstaande woning}>,<{Woonhuis}>,1971-1980,1,NaN,09F114F5C5EC061F6230349892132149,3,NaN,NaN,4388064,62,2018-11-12
1,4388200,2018-09-24,7481LK,209000.0,Verrassend ruime tussenwoning nabij het centru...,<{eengezinswoning}> <{tussenwoning}>,<{Woonhuis}>,1980,1,148.0,6A91BF7DB06A8DF2C9A89064F28571E7,5,1.0,B,4388200,136,2018-08-30
2,4399344,2018-08-02,1068MS,267500.0,- ENGLISH TRANSLATION - \n\nOn the 21st of Sep...,<{tussenverdieping}> (<{appartement}>),<{Appartement}>,2001-2010,0,NaN,E983FEDC63D87BF61AE952D181C8FD17,3,NaN,NaN,4399344,70,2018-11-23
3,4400638,2018-08-04,5628EN,349000.0,Wonen in een zeer royaal bemeten geschakelde 2...,<{eengezinswoning}> <{geschakelde 2-onder-1-ka...,<{Woonhuis}>,1973,1,244.0,02BC26608B8B1A0888D3612AC7A5DB5C,5,NaN,NaN,4400638,144,2018-12-14
4,4401765,2018-08-05,7731TV,495000.0,Landgoed Junne is een eeuwenoud landgoed en li...,<{woonboerderij}> <{vrijstaande woning}>,<{Woonhuis}>,1900,0,4500.0,F56B2705CE24B8D78A68481ED1B276CB,8,1.0,NaN,4401765,323,2018-12-06


### Preparing the data for tokenization

In [4]:
# dropping columns that are not needed for the analysis
df = df_original.drop(['kantoor_naam_MD5hash', 'globalId.1', 'perceelOppervlakte', 'aantalBadkamers', 'energielabelKlasse', 'publicatieDatum', 'datum_ondertekening', 'postcode', 'categorieObject', 'koopPrijs', 'soortWoning', 'bouwjaar', 'indTuin', 'aantalKamers', 'oppervlakte' ], 1)
df = df.rename(columns={'volledigeOmschrijving': 'description', 'globalId': 'id'})
df.head()

C:\Users\Joel\AppData\Local\Temp/ipykernel_14236/1254229570.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df_original.drop(['kantoor_naam_MD5hash', 'globalId.1', 'perceelOppervlakte', 'aantalBadkamers', 'energielabelKlasse', 'publicatieDatum', 'datum_ondertekening', 'postcode', 'categorieObject', 'koopPrijs', 'soortWoning', 'bouwjaar', 'indTuin', 'aantalKamers', 'oppervlakte' ], 1)


,id,description
0,4388064,"Ruimte, vrijheid, en altijd het water en de we..."
1,4388200,Verrassend ruime tussenwoning nabij het centru...
2,4399344,- ENGLISH TRANSLATION - \n\nOn the 21st of Sep...
3,4400638,Wonen in een zeer royaal bemeten geschakelde 2...
4,4401765,Landgoed Junne is een eeuwenoud landgoed en li...


In [5]:
# first take the text and transform it to a list, because it is easier to work with
text = df['description'].tolist()

In [6]:
# next, this command will load the dutch language model of spacy (the website), which we have previously installed and setting it to a variable
nlp = spacy.load("nl_core_news_sm")
type(nlp)

spacy.lang.nl.Dutch

In [8]:
# perform tokenization In our application, however, we will only need the tokenization feature; 
# Hence we will disable the other default features of the spacy pipeline and only run the rule-based tokenizer using nlp.make_doc. 
# This will also be a lot faster than using the statistical models. We can now tokenize all the documents.
# normal version gives error: docs = [nlp.make_doc(x) for x in text] 
docs = [nlp.make_doc(x) for x in text]

### Removing stopwords, numbers, punctuation and creating tokens

In [9]:
# words without 'real' meaning
stopwords = nlp.Defaults.stop_words
# to add words to this list
nlp.Defaults.stop_words |= {"doordat","m²", "x", "v.v", "o", "\n\n", "ca.", "\n\n\n", "\t\t", "\t"}
print(stopwords)

{'sindsdien', 'omver', 'zouden', 'onder', 'iets', 'sommige', 'zal', 'later', 'vaak', 'geweest', 'voort', 'hun', 'over', 'zou', 'hoewel', 'mijzelf', 'bent', 'm²', 'was', 'enkel', 'onszelf', 'veeleer', 'uwe', 'anders', 'kunt', 'erdoor', 'toch', 'beneden', 'meesten', 'steeds', 'jouw', 'al', 'jou', 'omlaag', 'rond', 'zulke', 'bepaald', 'even', 'weer', 'wezen', 'moeten', 'vroeg', 'vooral', 'ik', 'betere', 'min', 'precies', 'anderzijds', 'eigen', 'zijne', 'anderen', 'dat', 'zekere', 'ben', 'zodra', 'ca.', 'naar', 'die', 'niet', 'enkele', 'alles', 'dien', 'daarna', 'geworden', 'waren', 'elke', 'gewoon', 'net', 'ons', 'zij', 'zijn', 'vooraf', 'gewoonweg', 'omstreeks', 'reeds', 'hierboven', '\t', 'hadden', 'eerst', 'gehad', 'vanuit', 'u', 'werd', 'voorheen', 'uw', 'eveneens', 'me', 'des', 'doch', 'inzake', 'hen', 'wordt', 'behalve', 'mede', 'enz', 'op', 'ook', 'mochten', 'doen', 'om', 'te', 'ongeveer', 'zeer', 'jezelf', 'weinig', 'juist', 'eerder', 'jijzelf', 'veel', 'wat', 'moesten', 'vervolge

In [10]:
# the following lines of code illustrate how to check if a string is all composed of digits and punctuation using *regex*
digit_re = re.compile('^([0-9]|[\\.,])*$')

In [11]:
# loop to clean the data. First create list
# first loop is in docs: this is a list of all descriptions. One aspect of the list is a whole description. 
# second loop checks that created variable for stopwords, punctionation and regex and appends if it is not one of these, it adds it to the list new tokens
# if it has done it for the full descriptions, the list will be added to tokens_cleaned
tokens_cleaned = []
for doc in docs:
    new_tokens = []
    for token in doc:
        if not token.is_stop and not token.like_num and not token.is_space and not token.is_punct and not re.match(digit_re, token.text):
            new_tokens.append(token.text) # append the text only because we don't need the spacy tokens anymore from now on
    tokens_cleaned.append(new_tokens)

In [12]:
tokens_cleaned[0]

['Ruimte',
 'vrijheid',
 'water',
 'weides',
 'heen',
 'tikkeltje',
 'romantiek',
 'nostalgie',
 'voortdurende',
 'vakantiegevoel',
 'rust',
 'leven',
 'woonark',
 'nét',
 'ark',
 'voelt',
 'echt',
 'wonen',
 'Stilletjes',
 'deinend',
 'kabbelend',
 'water',
 'zwemmende',
 'eendjes',
 'bieden',
 'kans',
 'gaan',
 'ervaren',
 'Algemeen',
 'Rustig',
 'gelegen',
 'breed',
 'vaarwater',
 'dorp',
 'supermarkt',
 'snackbar',
 'café-restaurant',
 'Landelijke',
 'omgeving',
 'Omsloten',
 'tuin',
 'm2',
 'berging',
 'zuidoosten',
 'gebruiksvergoeding',
 'gemeente',
 'Ligplaatscanon',
 'via',
 'Hollands',
 'Noorderkwartier',
 'Garage',
 'parkeerruimte',
 'wegzijde',
 'Omschrijving',
 'gelijkvloers',
 'charmante',
 'woonark',
 'tuin',
 'stapt',
 'loopbrug',
 'ark',
 'ervaart',
 'direct',
 'bijzondere',
 'sfeer',
 'wonen',
 'water',
 'corridor',
 'loopt',
 'gehele',
 'lengte',
 'ark',
 'Links',
 'knusse',
 'woonkamer',
 'daarachter',
 'keuken',
 'koelkast',
 'wasemkap',
 'elektrische',
 'kookplaat

### Creating a matrix of the individual words and the corpus

In [13]:
# create matrix with N x M where N is all the lists/beschrijvingen/individual documents and M the individual words in the corpus (all words)
# we need to pass a dummy function to the tokenizer and preprocessor parameters of count vectorizer because we already calculated our tokens
def dummy(doc):
    return doc

count_vectorizer = CountVectorizer(
    tokenizer=dummy,
    preprocessor=dummy,
    )

frequency_matrix = count_vectorizer.fit_transform(tokens_cleaned)

In [14]:
count_vectorizer.get_feature_names_out()[:200]

array(['"DE', '$', "'N", "'S", "'m", "'n", "'ns", "'s", '(1.89', '(2.51',
       '(3.45', '(4.36', '(4.41', '(6.05', '(9.03', '(=', '(Albert',
       '(Conform', '(mogelijkheid', '(vergunningszone', '(voorheen',
       '(voorzien', ')-toevoer', '+', '+0,5', '+0,8*0,8', '+0.5', '+0.50',
       '+0.81', '+1', '+1),(entree', '+1,20', '+1,48x2', '+1,5', '+1.15',
       '+1.48', '+1.5', '+1.50', '+1.60', '+1/2', '+1/slaapgedeelte',
       '+10', '+100', '+11', '+119', '+12m2', '+144', '+16', '+1604',
       '+1623', '+17m2', '+18', '+1922', '+1e', '+1½', '+2', '+2,05',
       '+2,5', '+2,70', '+2.05*0.95', '+2.13x2.78', '+2.40', '+2.8',
       '+20', '+200', '+220', '+25', '+2e', '+2x', '+3', '+3,20', '+3.75',
       '+3.78', '+31', '+31(0)115', '+31(0)115562054', '+31(0)20',
       '+31(0)88-200-2000', '+31653146364', '+39', '+4', '+40m2', '+4860',
       '+4m.brede', '+5', '+51', '+55', '+6340', '+80m2', '-"De',
       '-"gemak', "-'30", "-'30-bebouwing", "-'45", "-'Buiten",
       "-'s-G

In [16]:
# doing it for only 5000 columns
count_vectorizer = CountVectorizer(
    tokenizer=dummy,
    preprocessor=dummy,
    max_features=5000 # add the max_features argument
    )

frequency_matrix = count_vectorizer.fit_transform(tokens_cleaned)

In [17]:
frequency_matrix

<211617x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 28138459 stored elements in Compressed Sparse Row format>

In [18]:
# Some words are not so important for a description because they occur often such as house. To take account for this use tf-df transformer to give words weights.
# words that occur often get smaller weights
tfd_idf_trans = TfidfTransformer()
tf_idf_matrix = tfd_idf_trans.fit_transform(frequency_matrix)

In [19]:
tf_idf_matrix

<211617x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 28138459 stored elements in Compressed Sparse Row format>

In [20]:
# we use 10 as the n_components here
model = NMF(n_components=10, init='nndsvd')

# fit the model
model.fit(X=tf_idf_matrix)

NMF(init='nndsvd', n_components=10)

In [21]:
# the model.components_ matrix is the H matrix mapping topics to word weights
model.components_
model.components_.shape

array([[0.        , 0.12118971, 0.01289418, ..., 0.        , 0.02020981,
        0.1243055 ],
       [0.00236275, 0.0967374 , 0.00069056, ..., 0.        , 0.00176701,
        0.15108646],
       [0.00090557, 0.04383746, 0.        , ..., 0.        , 0.        ,
        0.03723335],
       ...,
       [0.        , 0.12438954, 0.02775716, ..., 0.        , 0.00647816,
        0.21608534],
       [0.        , 0.1404377 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10207753, 0.10624197, 0.        , ..., 0.09913807, 0.02456108,
        0.        ]])

(10, 5000)

### Creating an object for the topic modelling and executing it on the prepared dataset

In [22]:
def get_topics(model, n_top_words):
    '''This function takes a fitted NMF factorization model, and a n_top_words parameter.
    It then produces a dataframe where the columns are the topics that have been learned, and the
    rows are the top words that define the topic, ranked by their coefficients.
    '''

    feature_names = count_vectorizer.get_feature_names_out()
    d = {}
    for i in range(model.n_components):
        # model.components_ matrix contains the matrix H where topics are 
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feature_names[key] for key in words_ids]
        d['Topic # ' + '{:02d}'.format(i+1)] = words
    return pd.DataFrame(d)

In [23]:
get_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,woning,the,m,appartement,woningen,m²,Meetinstructie,ca.,voorzien,woning
1,verdieping,and,x,balkon,m2,circa,particuliere,m2,wanden,ruimte
2,achtertuin,with,Slaapkamer,€,appartementen,garage,NEN2580,m.,plafond,tuin
3,ruime,a,m.,complex,type,±,koopovereenkomst,x,toegang,huis
4,2e,The,circa,per,fase,m³,verschillen,v.v,laminaatvloer,grote
5,toilet,to,afm,maand,standaard,woning,verkoper,Slaapkamer,vloer,garage
6,gelegen,floor,Woonkamer,gelegen,wonen,perceel,meten,m3,stucwerk,ligt
7,voorzien,on,Badkamer,lift,hoekwoningen,bijkeuken,koper,€,Slaapkamer,biedt
8,slaapkamers,room,achterzijde,uitzicht,sanitair,grond,meting,Inhoud,woning,woonkamer
9,1e,apartment,gelegen,berging,circa,Inhoud,gebruiksoppervlakte,Woonoppervlakte,afgewerkt,heerlijk


In [24]:
document_topics = model.transform(X=tf_idf_matrix)
document_topics = pd.DataFrame(document_topics)
main_topic = document_topics.idxmax(axis=1)
topics = ["Topic " + str(x) for x in range(1, 11)]
main_topic = main_topic.apply(lambda x: topics[x])

In [26]:
df['main_topic'] = main_topic

### Merging the dataframes together to one final dataframe

In [57]:
housing_data_complete = df.merge(df_original, how='inner', on=df['id'])

In [58]:
housing_data_complete.head()
housing_data_complete.tail()

,key_0,id,description,main_topic,globalId,publicatieDatum,postcode,koopPrijs,volledigeOmschrijving,soortWoning,...,bouwjaar,indTuin,perceelOppervlakte,kantoor_naam_MD5hash,aantalKamers,aantalBadkamers,energielabelKlasse,globalId.1,oppervlakte,datum_ondertekening
0,4388064,4388064,"Ruimte, vrijheid, en altijd het water en de we...",Topic 10,4388064,2018-07-31,1774PG,139000.0,"Ruimte, vrijheid, en altijd het water en de we...",<{woonboot}> <{vrijstaande woning}>,...,1971-1980,1,NaN,09F114F5C5EC061F6230349892132149,3,NaN,NaN,4388064,62,2018-11-12
1,4388200,4388200,Verrassend ruime tussenwoning nabij het centru...,Topic 10,4388200,2018-09-24,7481LK,209000.0,Verrassend ruime tussenwoning nabij het centru...,<{eengezinswoning}> <{tussenwoning}>,...,1980,1,148.0,6A91BF7DB06A8DF2C9A89064F28571E7,5,1.0,B,4388200,136,2018-08-30
2,4399344,4399344,- ENGLISH TRANSLATION - \n\nOn the 21st of Sep...,Topic 2,4399344,2018-08-02,1068MS,267500.0,- ENGLISH TRANSLATION - \n\nOn the 21st of Sep...,<{tussenverdieping}> (<{appartement}>),...,2001-2010,0,NaN,E983FEDC63D87BF61AE952D181C8FD17,3,NaN,NaN,4399344,70,2018-11-23
3,4400638,4400638,Wonen in een zeer royaal bemeten geschakelde 2...,Topic 3,4400638,2018-08-04,5628EN,349000.0,Wonen in een zeer royaal bemeten geschakelde 2...,<{eengezinswoning}> <{geschakelde 2-onder-1-ka...,...,1973,1,244.0,02BC26608B8B1A0888D3612AC7A5DB5C,5,NaN,NaN,4400638,144,2018-12-14
4,4401765,4401765,Landgoed Junne is een eeuwenoud landgoed en li...,Topic 6,4401765,2018-08-05,7731TV,495000.0,Landgoed Junne is een eeuwenoud landgoed en li...,<{woonboerderij}> <{vrijstaande woning}>,...,1900,0,4500.0,F56B2705CE24B8D78A68481ED1B276CB,8,1.0,NaN,4401765,323,2018-12-06


,key_0,id,description,main_topic,globalId,publicatieDatum,postcode,koopPrijs,volledigeOmschrijving,soortWoning,...,bouwjaar,indTuin,perceelOppervlakte,kantoor_naam_MD5hash,aantalKamers,aantalBadkamers,energielabelKlasse,globalId.1,oppervlakte,datum_ondertekening
211612,4613507,4613507,Ruime eengezinswoning aangeboden aan de rand v...,Topic 8,4613507,2018-12-31,9791GD,159000.0,Ruime eengezinswoning aangeboden aan de rand v...,<{eengezinswoning}> <{tussenwoning}>,...,1974,1,195.0,AB928B88F4860C427BB7C6A27D6F2642,5,NaN,NaN,4613507,116,2019-03-07
211613,4613532,4613532,Nabij centrum van Hoensbroek met al haar voorz...,Topic 1,4613532,2018-12-31,6431GT,179000.0,Nabij centrum van Hoensbroek met al haar voorz...,<{eengezinswoning}> <{geschakelde 2-onder-1-ka...,...,1995,1,160.0,4271E4C370DF497EE3BB23B3AED3E292,4,1.0,NaN,4613532,84,2019-01-17
211614,4613537,4613537,Ook de kans aangrijpen om te mogen wonen in de...,Topic 3,4613537,2018-12-31,4051EW,391500.0,Ook de kans aangrijpen om te mogen wonen in de...,<{bungalow}> <{vrijstaande woning}> (<{semi-bu...,...,1948,1,465.0,AB928B88F4860C427BB7C6A27D6F2642,5,NaN,NaN,4613537,99,2019-08-03
211615,4613566,4613566,"BIEDEN VANAF € 375.000,= kosten koper\nHet uit...",Topic 9,4613566,2018-12-31,4043GG,375000.0,"BIEDEN VANAF € 375.000,= kosten koper\nHet uit...",<{eengezinswoning}> <{2-onder-1-kapwoning}>,...,2006,1,300.0,59A3951B92EB2334DC7295CEEA0FB547,7,1.0,A,4613566,172,2019-04-08
211616,4577856,4577856,Type Lariks\n\nDe twee-onder-een-kapwoningen t...,Topic 5,4577856,2018-12-12,5036XA,395000.0,Type Lariks\n\nDe twee-onder-een-kapwoningen t...,<{eengezinswoning}> <{2-onder-1-kapwoning}>,...,2019,1,NaN,A3F8C9345E47A902FC0CE06865993E9D,6,1.0,NaN,4577856,156,2019-01-30


### Exporting the dataset

In [ ]:
housing_data_complete.to_csv('housing_topic_modelling.csv')